In [1]:
import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4

Things to think about:
* what if control day not in temperature data set? (solution is to add more temperature data)
* what if control day is actually a heat wave?
* what if control day is close to a heat wave?

In [2]:
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd



## Load heatwaves and get initial control dates

In [3]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/nigeria/THI_nigeria_90_2022.csv')

In [4]:
heatwaves['month'].unique()

array([3, 4, 2])

In [5]:
heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [6]:
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)


In [7]:
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [8]:
heatwaves_active = heatwaves[heatwaves['heatwave'] == 1]
heat_days = heatwaves_active.groupby('location')['datetime'].unique().reset_index().set_index('location')

In [9]:
heat_days = heat_days.to_dict()['datetime']

In [10]:
heatwaves_active.reset_index(inplace=True)

In [11]:
full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/THI/nigeria/daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df.month.unique()
full_df = full_df[~full_df['month'].isin([5])]

full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]


In [12]:
full_df['temp_pct'] = full_df.groupby('location')['thi'].rank(pct=True)

In [13]:
heatwaves_active['num_iters'] = 'test'

In [15]:

for index, row in heatwaves_active.iterrows():
    subset = full_df[full_df['location'] == row['location']]
    control_day = heatwaves_active.iloc[index, -3]
    i = 0
    try:
        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):


            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] - timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]

    except:
        heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
        control_day = heatwaves_active.iloc[index, -3]

        while ((subset[subset['datetime'] == control_day]['temp_pct'].values[0]>= 0.8) | 
               (subset[(subset['datetime'] == control_day)]['temp_pct'].values[0]<= 0.2)):
        
            heatwaves_active.iloc[index, -3] = heatwaves_active.iloc[index, -3] + timedelta(days=7)
            control_day = heatwaves_active.iloc[index, -3]
            
    print(index / len(heatwaves_active))
            


0.0
0.0001087547580206634
0.0002175095160413268
0.0003262642740619902
0.0004350190320826536
0.000543773790103317
0.0006525285481239804
0.0007612833061446438
0.0008700380641653072
0.0009787928221859706
0.001087547580206634
0.0011963023382272975
0.0013050570962479609
0.0014138118542686242
0.0015225666122892876
0.0016313213703099511
0.0017400761283306145
0.0018488308863512778
0.001957585644371941
0.0020663404023926048
0.002175095160413268
0.0022838499184339315
0.002392604676454595
0.002501359434475258
0.0026101141924959217
0.0027188689505165853
0.0028276237085372484
0.002936378466557912
0.003045133224578575
0.0031538879825992387
0.0032626427406199023
0.0033713974986405654
0.003480152256661229
0.0035889070146818925
0.0036976617727025557
0.0038064165307232192
0.003915171288743882
0.004023926046764546
0.0041326808047852095
0.004241435562805873
0.004350190320826536
0.004458945078847199
0.004567699836867863
0.0046764545948885265
0.00478520935290919
0.004893964110929853
0.005002718868950516
0.0

In [16]:
heatwaves_active['control_day'].min()

Timestamp('2022-02-07 00:00:00')

In [17]:
heatwaves_active[['datetime', 'control_day']]

,datetime,control_day
0,2022-03-07,2022-02-28
1,2022-03-08,2022-02-22
2,2022-03-07,2022-02-28
3,2022-03-08,2022-02-22
4,2022-03-29,2022-03-15
...,...,...
9190,2022-04-14,2022-03-31
9191,2022-04-21,2022-03-31
9192,2022-04-22,2022-04-15
9193,2022-04-23,2022-04-16


In [18]:
valid = heatwaves_active.merge(full_df[['location', 'datetime', 'temp_pct']], left_on=['location', 'control_day'], right_on=['location', 'datetime'])

In [19]:
valid['temp_pct'].describe()

count    9195.000000
mean        0.639552
std         0.106245
min         0.206402
25%         0.565121
50%         0.660044
75%         0.725166
max         0.799117
Name: temp_pct, dtype: float64

In [20]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves_active.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/paper2/nigeria/THI_heatwave_control_90.csv')